# Connexion à l'API France Travail

URL : 

https://francetravail.io/data/api/offres-emploi

https://francetravail.io/data/api/offres-emploi/documentation#/

## Imports

In [67]:
import http.client
import requests
import json
import pandas as pd

In [68]:
# ---------------------------
# CONFIGURATION
# ---------------------------
# France Travail (ex-Pôle emploi)
# FT_CLIENT_ID = Defined in .env
# FT_CLIENT_SECRET = Defined in .env
# FT_SCOPE = Defined in .env

# Paramètres de recherche
JOB_QUERY = "data analyst"
LOCATION = "Paris"
MAX_PAGES = 3   # Limiter le nombre de pages récupérées

# ---------------------------
# AUTH FRANCE TRAVAIL
# ---------------------------
def get_ft_token():
    url = "https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=/partenaire"
    data = {
        "grant_type": "client_credentials",
        "client_id": FT_CLIENT_ID,
        "client_secret": FT_CLIENT_SECRET,
        "scope": FT_SCOPE,
    }
    r = requests.post(url, data=data)
    r.raise_for_status()
    return r.json()["access_token"]

# ---------------------------
# API CALL FRANCE TRAVAIL
# ---------------------------
def fetch_france_travail_jobs(token, max_pages=MAX_PAGES):
    headers = {"Authorization": f"Bearer {token}"}
    all_jobs = []
    for page in range(1, max_pages + 1):
        url = f"https://api.francetravail.io/partenaire/offresdemploi/v2/offres/search"
        params = {
            "motsCles": JOB_QUERY,
            "lieux": LOCATION,
            "range": f"{(page-1)*50}-{page*50-1}"  # pagination par blocs de 50
        }
        r = requests.get(url, headers=headers, params=params)
        r.raise_for_status()
        data = r.json()
        offres = data.get("resultats", [])
        for o in offres:
            all_jobs.append({
                "source": "France Travail",
                "titre": o.get("intitule"),
                "entreprise": o.get("entreprise", {}).get("nom"),
                "lieu": o.get("lieuTravail", {}).get("libelle"),
                "date": o.get("dateCreation"),
                "url": o.get("origineOffre", {}).get("urlOrigine"),
            })
    return all_jobs

# ---------------------------
# MAIN
# ---------------------------
if __name__ == "__main__":
    print("Authentification France Travail...")
    token = get_ft_token()

    print("Récupération des offres France Travail...")
    ft_jobs = fetch_france_travail_jobs(token)

    df = pd.DataFrame(ft_jobs)
    print(df.head())

    # Export vers CSV
    df.to_csv("../data/raw_data/offres_emploi.csv", index=False, encoding="utf-8")
    print(f"{len(ft_jobs)} offres uniques exportées dans offres_emploi.csv ✅")


Authentification France Travail...
Récupération des offres France Travail...
           source                                    titre  \
0  France Travail                       Data analyst (H/F)   
1  France Travail                       Data Analyst (h/f)   
2  France Travail  Data Analyst Technico-fonctionnel (H/F)   
3  France Travail                 Data analyst (F/H) (H/F)   
4  France Travail                       Data analyst (F/H)   

           entreprise              lieu                      date  \
0  LEADER GOLBEY 2029       88 - Épinal  2025-09-13T13:57:01.416Z   
1       ADECCO FRANCE    13 - Marignane  2025-09-09T16:27:22.779Z   
2              AFSTER      69 - LYON 01  2025-09-05T15:12:07.043Z   
3      AESIO MUTUELLE      26 - Valence  2025-09-04T13:51:37.276Z   
4            EXPECTRA  34 - Montpellier  2025-09-02T14:41:39.561Z   

                                                 url  
0  https://candidat.francetravail.fr/offres/reche...  
1  https://candidat.franc